# 基于K-近邻的车牌号识别

## 一、案例简介

图像的智能处理一直是人工智能领域广受关注的一类技术，代表性的如人脸识别与 CT 肿瘤识别，在人工智能落地的进程中发挥着重要作用。其中车牌号识别作为一个早期应用场景，已经融入日常生活中，为我们提供了诸多便利，在各地的停车场和出入口都能看到它的身影。车牌号识别往往分为字符划分和字符识别两个子任务，本案例我们将关注字符识别的任务，尝试用 K-NN 的方法对分割好的字符图像进行自动识别和转化。

## 二、数据概览
本次我们使用已经分割好的车牌图片作为数据集，包括数字 0-9、字母 A-Z（不包含 O 和 I）以及省份简称共 65 个类，编号从 0 到 64。数据已经分成了训练集和测试集，里面的文件夹用 label 编号命名，一个文件夹下的所有图片都属于该文件夹对应的类，每个图片都是 20 * 20 的二值化灰度图。

下面演示一下如何借助 PIL 库将图片转化为向量：

In [1]:
from PIL import Image
img = Image.open('data/train/0/4-3.jpg')  # 打开图片
img  # 显示图片from PIL import Image
img = Image.open('data/train/0/4-3.jpg')  # 打开图片
img  # 显示图片

In [2]:
import numpy as np
pixels = np.array(img)  # 转化为 numpy 矩阵
print(pixels.shape)
#print(pixels)

(20, 20)


## 三、模型构建

In [3]:
from PIL import Image
import numpy as np
import pandas as pd
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [4]:
train_path = "./data/train/"
test_path = "./data/test/"
train_x = []
train_y = []
test_x = []
test_y = []
for root, ds, fs in os.walk(train_path):
    for d in ds:
        dir = os.path.join(root, d)
        for l in os.listdir(dir):
            path = os.path.join(dir, l)
            img = Image.open(path)
            pixels = np.array(img).reshape(-1)
            #pixels = np.array(img)
            train_x.append(pixels)
            train_y.append(d)
            
for root, ds, fs in os.walk(test_path):
    for d in ds:
        dir = os.path.join(root, d)
        for l in os.listdir(dir):
            path = os.path.join(dir, l)
            img = Image.open(path)
            pixels = np.array(img).reshape(-1)
            #pixels = np.array(img)
            test_x.append(pixels)
            test_y.append(d)

train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)
print(train_x.shape, test_x.shape)

(15954, 400) (4665, 400)


In [5]:
print(train_x[0])

[  1   0   1   0   0   2  26 139 254 255 254 254 141  27   1   2   0   0
   0   0   0   2   2   0   1  95 254 255 255 250 253 255 250 253  27   0
   0   0   0   0   0   3   0   0 119 255 251 190 126 128 131 162 255 254
 140   0   0   0   0   0   0   0   2   4 229 252 203  39   0   1   0  23
 166 251 255   3   0   0   0   0   3   0   1   0 229 255 177   1   0   8
   0   1 130 253 255   0   0   0   0   0   0   2   4   0 232 252 182   0
   3   0   2   1 126 255 252 233   0   0   0   0   3   3   0   4 227 254
 182   3   0   1   0   2 122 255 255 230   0   0   0   0   0   0   5   0
 229 255 176   0   0   2   0   0  64 219 254 227   0   0   0   0   0   0
   1   0 230 254 180   0   5   0   1   0   0 180 254 229   0   0   0   0
   0   0   1   1 228 253 179   2   0   7   1   0 119 247 253 232   0   0
   0   0   1   1   0   1 230 254 176   0   2   1   0   5 127 254 253 231
   0   0   0   0   1   1   0   0 231 253 177   1   0   2   0   0  91 233
 255 229   0   0   0   0   0   0   1   0 230 252 18

### n=1~10, weight=uniform
观察k值从1到10，使用平权。

In [6]:
for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=i, weights = 'uniform', n_jobs=-1,algorithm='kd_tree')
    knn.fit(train_x,train_y)
    predict_y = knn.predict(test_x)
    acc = accuracy_score(predict_y, test_y)
    print("n=%d,accuracy score is : %.6f" %(i,acc))

n=1,accuracy score is : 0.716827
n=2,accuracy score is : 0.721972
n=3,accuracy score is : 0.703108
n=4,accuracy score is : 0.707395
n=5,accuracy score is : 0.696892
n=6,accuracy score is : 0.696463
n=7,accuracy score is : 0.695606
n=8,accuracy score is : 0.692390
n=9,accuracy score is : 0.691318
n=10,accuracy score is : 0.691104


### n=1~10, weight=distance, p=2(Euclidean distance)
即观察k值从1到10，使用距离加权，使用欧氏距离  sqrt(sum((x - y)^2))

In [7]:
for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=i, weights = 'distance', n_jobs=-1, algorithm='kd_tree')
    knn.fit(train_x,train_y)
    predict_y = knn.predict(test_x)
    acc = accuracy_score(predict_y, test_y)
    print("n=%d, accuracy score is : %.6f" %(i,acc))

n=1, accuracy score is : 0.716827
n=2, accuracy score is : 0.716827
n=3, accuracy score is : 0.710397
n=4, accuracy score is : 0.707824
n=5, accuracy score is : 0.701608
n=6, accuracy score is : 0.699250
n=7, accuracy score is : 0.699250
n=8, accuracy score is : 0.695820
n=9, accuracy score is : 0.694962
n=10, accuracy score is : 0.694319


也是欧氏距离，测试工具用法

In [8]:
for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=i, weights = 'distance', n_jobs=-1, algorithm='kd_tree', p=2, metric='minkowski')
    knn.fit(train_x,train_y)
    predict_y = knn.predict(test_x)
    acc = accuracy_score(predict_y, test_y)
    print("n=%d, accuracy score is : %.6f" %(i,acc))

n=1, accuracy score is : 0.716827
n=2, accuracy score is : 0.716827
n=3, accuracy score is : 0.710397
n=4, accuracy score is : 0.707824
n=5, accuracy score is : 0.701608
n=6, accuracy score is : 0.699250
n=7, accuracy score is : 0.699250
n=8, accuracy score is : 0.695820
n=9, accuracy score is : 0.694962
n=10, accuracy score is : 0.694319


### n=1~10, weight=distance, p=1(Manhatan distance)
使用曼哈顿距离观察下 sum(|x - y|)

In [8]:
for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=i, weights = 'distance', n_jobs=-1, algorithm='kd_tree',p=1,metric='minkowski')
    knn.fit(train_x,train_y)
    predict_y = knn.predict(test_x)
    acc = accuracy_score(predict_y, test_y)
    print("n=%d, accuracy score is : %.6f" %(i,acc))

n=1, accuracy score is : 0.714469
n=2, accuracy score is : 0.714469
n=3, accuracy score is : 0.706109
n=4, accuracy score is : 0.697749
n=5, accuracy score is : 0.697535
n=6, accuracy score is : 0.694962
n=7, accuracy score is : 0.689389
n=8, accuracy score is : 0.686388
n=9, accuracy score is : 0.684887
n=10, accuracy score is : 0.684244


### 使用闵可夫斯基距离  sum(|x - y|^p)^(1/p); p=3 (3次方的速度真的超级慢。。。）

In [10]:
for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=i, weights = 'distance', n_jobs=-1, algorithm='kd_tree',p=3, metric='minkowski')
    knn.fit(train_x,train_y)
    predict_y = knn.predict(test_x)
    acc = accuracy_score(predict_y, test_y)
    print("n=%d, accuracy score is : %.6f" %(i,acc))

n=1, accuracy score is : 0.711254
n=2, accuracy score is : 0.711254
n=3, accuracy score is : 0.707610
n=4, accuracy score is : 0.703537
n=5, accuracy score is : 0.703108
n=6, accuracy score is : 0.701608
n=7, accuracy score is : 0.696892
n=8, accuracy score is : 0.693676
n=9, accuracy score is : 0.691318
n=10, accuracy score is : 0.691533


### 使用切比雪夫距离  	 max(|x - y|)    
效果不好

In [11]:
for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=i, weights = 'distance', n_jobs=-1, algorithm='kd_tree', metric='chebyshev')
    knn.fit(train_x,train_y)
    predict_y = knn.predict(test_x)
    acc = accuracy_score(predict_y, test_y)
    print("n=%d, accuracy score is : %.6f" %(i,acc))

n=1, accuracy score is : 0.449946
n=2, accuracy score is : 0.465809
n=3, accuracy score is : 0.453805
n=4, accuracy score is : 0.447374
n=5, accuracy score is : 0.432583
n=6, accuracy score is : 0.428510
n=7, accuracy score is : 0.425723
n=8, accuracy score is : 0.424437
n=9, accuracy score is : 0.413934
n=10, accuracy score is : 0.414791


### 通过以上使用不同方法（n值，距离加权）后对数据的观察，除了切比雪夫距离比较差，其他结果并没有太显著的变化。n值取1~2为最优。


### 觉得识别率太低，对比下其他数据集的准确率，试图分析是算法原因还是数据集原因
### 导入手写数字数据集

In [9]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
digits = datasets.load_digits()
print(digits.data.shape)
train_x, test_x, train_y, test_y = train_test_split(digits.data, digits.target, test_size=0.2, random_state=8)
dknn = KNeighborsClassifier(n_neighbors=3, weights = 'distance', n_jobs=-1, algorithm='kd_tree', metric='minkowski')
dknn.fit(train_x, train_y)
predict_y = dknn.predict(test_x)
acc = accuracy_score(predict_y, test_y)
print("acc : %.6f" %(acc))

(1797, 64)
acc : 0.988889


In [10]:
print(type(digits.data))

<class 'numpy.ndarray'>


### 发现经典数据集digits的准确率很高，分析区别：  
1.牌照data set sample 的feature大小是 : 20 * 20，digit data set feature的大小是 : 8 * 8。可能牌照数据样本的矩阵太稀疏。  
2.整个训练样本很小，尝试调整牌照train set比例。  
3.牌照数据集训练样本复杂（数字，字母，区域），而digit训练样本单一，只有数字。  

### 直觉告诉我是第三个原因，将数据集按照数字，字母，区域划分，分别再训练

In [11]:
# 0~9 number, 10~33 A~Z, 34~65 zone
train_path = "./data/train/"
test_path = "./data/test/"
# number data 0~9
train_n_x = []
train_n_y = []
test_n_x = []
test_n_y = []
# alphabet data 10~33
train_alp_x = []
train_alp_y = []
test_alp_x = []
test_alp_y = []
# zone data 34~65
train_z_x = []
train_z_y = []
test_z_x = []
test_z_y = []

for root, ds, fs in os.walk(train_path):
    for d in ds:
        dir = os.path.join(root, d)
        for l in os.listdir(dir):
            path = os.path.join(dir, l)
            img = Image.open(path)
            pixels = np.array(img).reshape(-1)
            if int(d) in range(0,10):
                train_n_x.append(pixels)
                train_n_y.append(d)
            elif int(d) in range(10,34):
                train_alp_x.append(pixels)
                train_alp_y.append(d)
            else:
                train_z_x.append(pixels)
                train_z_y.append(d)
            
for root, ds, fs in os.walk(test_path):
    for d in ds:
        dir = os.path.join(root, d)
        for l in os.listdir(dir):
            path = os.path.join(dir, l)
            img = Image.open(path)
            pixels = np.array(img).reshape(-1)
            if int(d) in range(0,10):
                test_n_x.append(pixels)
                test_n_y.append(d)
            elif int(d) in range(10,34):
                test_alp_x.append(pixels)
                test_alp_y.append(d)
            else:
                test_z_x.append(pixels)
                test_z_y.append(d)

train_n_x = np.array(train_n_x)
train_n_y = np.array(train_n_y)
test_n_x = np.array(test_n_x)
test_n_y = np.array(test_n_y)
# alphabet data 10~33
train_alp_x = np.array(train_alp_x)
train_alp_y = np.array(train_alp_y)
test_alp_x = np.array(test_alp_x)
test_alp_y = np.array(test_alp_y)
# zone data 34~65
train_z_x = np.array(train_z_x)
train_z_y = np.array(train_z_y)
test_z_x = np.array(test_z_x)
test_z_y = np.array(test_z_y)
train_n_x.shape, train_n_y.shape, train_alp_x.shape, train_alp_y.shape, train_z_x.shape, train_z_y.shape

((7694, 400), (7694,), (5481, 400), (5481,), (2779, 400), (2779,))

### 单训练数字

In [12]:
for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=i, weights = 'distance', n_jobs=-1, algorithm='kd_tree')
    knn.fit(train_n_x,train_n_y)
    predict_y = knn.predict(test_n_x)
    acc = accuracy_score(predict_y, test_n_y)
    print("n=%d, accuracy score is : %.6f" %(i,acc))

n=1, accuracy score is : 0.989880
n=2, accuracy score is : 0.989880
n=3, accuracy score is : 0.988040
n=4, accuracy score is : 0.988960
n=5, accuracy score is : 0.988040
n=6, accuracy score is : 0.988040
n=7, accuracy score is : 0.986201
n=8, accuracy score is : 0.986201
n=9, accuracy score is : 0.986201
n=10, accuracy score is : 0.985281


### 单训练字母

In [13]:
for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=i, weights = 'distance', n_jobs=-1, algorithm='kd_tree')
    knn.fit(train_alp_x,train_alp_y)
    predict_y = knn.predict(test_alp_x)
    acc = accuracy_score(predict_y, test_alp_y)
    print("n=%d, accuracy score is : %.6f" %(i,acc))

n=1, accuracy score is : 0.794872
n=2, accuracy score is : 0.794872
n=3, accuracy score is : 0.776110
n=4, accuracy score is : 0.777048
n=5, accuracy score is : 0.767667
n=6, accuracy score is : 0.767667
n=7, accuracy score is : 0.765478
n=8, accuracy score is : 0.766104
n=9, accuracy score is : 0.757348
n=10, accuracy score is : 0.763290


### 单训练车牌城市区域

In [14]:
for i in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=i, weights = 'distance', n_jobs=-1, algorithm='kd_tree')
    knn.fit(train_z_x,train_z_y)
    predict_y = knn.predict(test_z_x)
    acc = accuracy_score(predict_y, test_z_y)
    print("n=%d, accuracy score is : %.6f" %(i,acc))

n=1, accuracy score is : 0.897368
n=2, accuracy score is : 0.897368
n=3, accuracy score is : 0.905263
n=4, accuracy score is : 0.915789
n=5, accuracy score is : 0.915789
n=6, accuracy score is : 0.921053
n=7, accuracy score is : 0.921053
n=8, accuracy score is : 0.926316
n=9, accuracy score is : 0.928947
n=10, accuracy score is : 0.923684


通过区分输入语言，单独训练后发现，数字的识别率非常好，城市中文字的识别率也ok，但字母的识别率很低，导致整体识别率不高。   
整体识别正确是需要以上三部分都识别正确。即3部分正确的概率相乘。   
P(all) = P(digit) * P(letter) * P(city) = 0.99 * 0.79 * 0.90 = 0.70389   
通过计算观察正确率与整体识别的正确率相当。  
如果最终的计算结果想要高于0.9，则每项结果平均需要大于0.965。